# 第8章: ニューラルネット

第7章で取り組んだポジネガ分類を題材として、ニューラルネットワークで分類モデルを実装する。なお、この章ではPyTorchやTensorFlow、JAXなどの深層学習フレームワークを活用せよ。

## 70. 単語埋め込みの読み込み

事前学習済み単語埋め込みを活用し、$|V| \times d_\rm{emb}$ の単語埋め込み行列$\pmb{E}$を作成せよ。ここで、$|V|$は単語埋め込みの語彙数、$d_\rm{emb}$は単語埋め込みの次元数である。ただし、単語埋め込み行列の先頭の行ベクトル$\pmb{E}_{0,:}$は、将来的にパディング（`<PAD>`）トークンの埋め込みベクトルとして用いたいので、ゼロベクトルとして予約せよ。ゆえに、$\pmb{E}$の2行目以降に事前学習済み単語埋め込みを読み込むことになる。

もし、Google Newsデータセットの[学習済み単語ベクトル](https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit?usp=sharing)（300万単語・フレーズ、300次元）を全て読み込んだ場合、$|V|=3000001, d_\rm{emb}=300$になるはずである（ただ、300万単語の中には、殆ど用いられない稀な単語も含まれるので、語彙を削減した方がメモリの節約になる）。

また、単語埋め込み行列の構築と同時に、単語埋め込み行列の各行のインデックス番号（トークンID）と、単語（トークン）への双方向の対応付けを保持せよ。

In [1]:
import numpy as np
import gensim

model = gensim.models.KeyedVectors.load_word2vec_format(
    "data\GoogleNews-vectors-negative300.bin",
    binary=True,
)

vocab_size = len(model.key_to_index) + 1
embedding_dim = model.vector_size
embedding_matrix = np.zeros((vocab_size, embedding_dim), dtype=np.float32)

token2id = {"<pad>": 0}
id2token = {0: "<pad>"}

for idx, word in enumerate(model.key_to_index):
    token2id[word] = idx + 1
    id2token[idx + 1] = word
    embedding_matrix[idx + 1] = model[word]

print(f"Embedding matrix shape: {embedding_matrix.shape}")

Embedding matrix shape: (3000001, 300)


## 71. データセットの読み込み

[General Language Understanding Evaluation (GLUE)](https://gluebenchmark.com/) ベンチマークで配布されている[Stanford Sentiment Treebank (SST)](https://dl.fbaipublicfiles.com/glue/data/SST-2.zip) をダウンロードし、訓練セット（train.tsv）と開発セット（dev.tsv）のテキストと極性ラベルと読み込み、全てのテキストをトークンID列に変換せよ。このとき、単語埋め込みの語彙でカバーされていない単語は無視し、トークン列に含めないことにせよ。また、テキストの全トークンが単語埋め込みの語彙に含まれておらず、空のトークン列となってしまう事例は、訓練セットおよび開発セットから削除せよ（このため、第7章の実験で得られた正解率と比較できなくなることに注意せよ）。

事例の表現方法は任意でよいが、例えば"contains no wit , only labored gags"がネガティブに分類される事例は、次のような辞書オブジェクトで表現すればよい。

```
{'text': 'contains no wit , only labored gags',
 'label': tensor([0.]),
 'input_ids': tensor([ 3475,    87, 15888,    90, 27695, 42637])}
```

この例では、`text`はテキスト、`label`は分類ラベル（ポジティブなら`tensor([1.])`、ネガティブなら`tensor([0.])`）、`input_ids`はテキストのトークン列をID列で表現している。

In [2]:
import os
import zipfile
import requests

# データセットのダウンロード
url = "https://dl.fbaipublicfiles.com/glue/data/SST-2.zip"
zip_path = "./data/SST-2.zip"

response = requests.get(url)
with open(zip_path, "wb") as f:
    f.write(response.content)
    
# ZIPファイルを解凍
with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall("./data")

In [3]:
import pandas as pd
import torch

train_path = "./data/SST-2/train.tsv"
dev_path = "./data/SST-2/dev.tsv"

def load_data(filepath):
    df = pd.read_csv(filepath, sep="\t", header=0)
    df.columns = ["text", "label"]
    df["label"] = df["label"].astype(int)
    return df

def make_dict(df):
    dataset = []
    for _, row in df.iterrows():
        text = row["text"]
        label = row["label"]
        
        # テキストをトークン化し、IDに変換
        words = text.split()
        input_ids = []
        
        for word in words:
            if word in token2id:
                input_ids.append(token2id[word])
        
        # 空のトークン列の事例は無視する
        if len(input_ids) > 0:
            dataset.append({
                "text": text,
                "label": torch.tensor([float(label)]),
                "input_ids": torch.tensor(input_ids)
            })
    
    return dataset

# データの読み込みと処理
train_df = load_data(train_path)
dev_df = load_data(dev_path)

train_dataset = make_dict(train_df)
dev_dataset = make_dict(dev_df)

print(f"Train dataset size: {len(train_dataset)}")
print(f"Dev dataset size: {len(dev_dataset)}")
print("\nExample:")
print(train_dataset[0])
   


Train dataset size: 66650
Dev dataset size: 872

Example:
{'text': 'hide new secretions from the parental units ', 'label': tensor([0.]), 'input_ids': tensor([  5785,     66, 113845,     18,     12,  15095,   1594])}


## 72. Bag of wordsモデルの構築

単語埋め込みの平均ベクトルでテキストの特徴ベクトルを表現し、重みベクトルとの内積でポジティブ及びネガティブを分類するニューラルネットワーク（ロジスティック回帰モデル）を設計せよ。

In [4]:
import torch
import torch.nn as nn

class LogisticRegression(nn.Module):
    def __init__(self, embedding_matrix, freeze_embedding=True):
        super().__init__()
        self.embedding = nn.Embedding.from_pretrained(
            torch.tensor(embedding_matrix, dtype=torch.float32),
            freeze=freeze_embedding,
        )
        self.linear = nn.Linear(embedding_matrix.shape[1], 1)
        
    def forward(self, input_ids):
        embedded = self.embedding(input_ids)
        mean_embedded = embedded.mean(dim=0)
        return self.linear(mean_embedded)
    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LogisticRegression(embedding_matrix, True).to(device)

## 73. モデルの学習

問題72で設計したモデルの重みベクトルを訓練セット上で学習せよ。ただし、学習中は単語埋め込み行列の値を固定せよ（単語埋め込み行列のファインチューニングは行わない）。また、学習時に損失値を表示するなど、学習の進捗状況をモニタリングできるようにせよ。

In [5]:
class LogisticRegression(nn.Module):
    def __init__(self, embedding_matrix, freeze_embedding=True):
        super().__init__()
        # 事前学習済みの埋め込み行列から nn.Embedding を作成
        # freeze=True なら重みを学習させない（凍結）
        self.embedding = nn.Embedding.from_pretrained(
            torch.tensor(embedding_matrix, dtype=torch.float32),
            freeze=freeze_embedding,
        )
        # 線形層：入力は埋め込みの次元数 → 出力は1
        self.linear = nn.Linear(embedding_matrix.shape[1], 1)

    def forward(self, input_ids):
        # 入力（トークンID列）を埋め込みベクトルに変換
        embedded = self.embedding(input_ids)  # [seq_len, emb_dim]
        # 埋め込みベクトルの平均を計算
        mean_embedded = embedded.mean(dim=0).unsqueeze(0)  # [1, emb_dim]
        return self.linear(mean_embedded)  # [1, 1]

model = LogisticRegression(embedding_matrix, True).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
criterion = nn.BCEWithLogitsLoss()

def train(model, dataset, optimizer, criterion, device):
    model.train()
    total_loss = 0.0
    for data in dataset:
        input_ids = data["input_ids"].to(device)
        labels = data["label"].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids)
        # ラベルを [1] → [1, 1] に変形して損失を計算
        loss = criterion(outputs, labels.unsqueeze(1))

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(dataset)

num_epochs = 5
for epoch in range(num_epochs):
    train_loss = train(model, train_dataset, optimizer, criterion, device)
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {train_loss:.4f}")

torch.save(model.state_dict(), "./model/No_73.pt")

Epoch 1/5, Loss: 0.6527
Epoch 2/5, Loss: 0.5942
Epoch 3/5, Loss: 0.5534
Epoch 4/5, Loss: 0.5241
Epoch 5/5, Loss: 0.5024


## 74. モデルの評価

問題73で学習したモデルの開発セットにおける正解率を求めよ。

In [6]:

import numpy as np
from sklearn.metrics import accuracy_score

def evaluate(model, dataset, device):
    model.eval() 

    y_preds = []  
    y_trues = [] 

    for data in dataset:
        input_ids = data["input_ids"].to(device)
        labels = data["label"].to(device)

        with torch.no_grad():  # 勾配計算を無効化（高速化・メモリ削減）
            outputs = model(input_ids)  
            probs = (
                torch.sigmoid(outputs).squeeze().cpu().numpy()
            )  # 確率に変換

            y_preds.append(probs)
            y_trues.append(labels.item())  # .item() で整数に変換

    y_preds = np.array(y_preds)
    y_trues = np.array(y_trues)

    binary_preds = (y_preds > 0.5).astype(int)
    accuracy = accuracy_score(y_trues, binary_preds)

    return accuracy

accuracy = evaluate(model, dev_dataset, device)
print(f"Validation Accuracy: {accuracy:.4f}")

Validation Accuracy: 0.7649


## 75. パディング

複数の事例が与えられたとき、これらをまとめて一つのテンソル・オブジェクトで表現する関数`collate`を実装せよ。与えられた複数の事例のトークン列の長さが異なるときは、トークン列の長さが最も長いものに揃え、0番のトークンIDでパディングをせよ。さらに、トークン列の長さが長いものから順に、事例を並び替えよ。

例えば、訓練データセットの冒頭の4事例が次のように表されているとき、

```
[{'text': 'hide new secretions from the parental units',
  'label': tensor([0.]),
  'input_ids': tensor([  5785,     66, 113845,     18,     12,  15095,   1594])},
 {'text': 'contains no wit , only labored gags',
  'label': tensor([0.]),
  'input_ids': tensor([ 3475,    87, 15888,    90, 27695, 42637])},
 {'text': 'that loves its characters and communicates something rather beautiful about human nature',
  'label': tensor([1.]),
  'input_ids': tensor([    4,  5053,    45,  3305, 31647,   348,   904,  2815,    47,  1276,  1964])},
 {'text': 'remains utterly satisfied to remain the same throughout',
  'label': tensor([0.]),
  'input_ids': tensor([  987, 14528,  4941,   873,    12,   208,   898])}]
```

`collate`関数を通した結果は以下のようになることが想定される。

```
{'input_ids': tensor([
    [     4,   5053,     45,   3305,  31647,    348,    904,   2815,     47,   1276,   1964],
    [  5785,     66, 113845,     18,     12,  15095,   1594,      0,      0,      0,      0],
    [   987,  14528,   4941,    873,     12,    208,    898,      0,      0,      0,      0],
    [  3475,     87,  15888,     90,  27695,  42637,      0,      0,      0,      0,      0]]),
 'label': tensor([
    [1.],
    [0.],
    [0.],
    [0.]])}
```


In [7]:
from pprint import pprint
import torch
import torch.nn.utils.rnn as rnn_utils

def collate(dataset):
    # トークン列の長さ（＝"input_ids"の要素数）でソート
    dataset = sorted(dataset, key=lambda x: len(x["input_ids"]), reverse=True)
    
    input_ids = [data["input_ids"] for data in dataset]
    labels = [data["label"] for data in dataset]
    
    # 最長のトークン列に合わせてパディング
    input_ids = rnn_utils.pad_sequence(input_ids, batch_first=True, padding_value=0)
    # ラベルはそのままスタック
    labels = torch.stack(labels)
    
    return {"input_ids": input_ids, "label": labels}

pprint(collate(train_dataset[:5]))

{'input_ids': tensor([[     4,   5053,     45,   3305,  31647,    348,    904,   2815,     47,
           1276,   1964],
        [     6,     12,   1445,  43789,     12,  10946,     76,  41349,     42,
              0,      0],
        [  5785,     66, 113845,     18,     12,  15095,   1594,      0,      0,
              0,      0],
        [   987,  14528,   4941,    873,     12,    208,    898,      0,      0,
              0,      0],
        [  3475,     87,  15888,     90,  27695,  42637,      0,      0,      0,
              0,      0]]),
 'label': tensor([[1.],
        [0.],
        [0.],
        [0.],
        [0.]])}


## 76. ミニバッチ学習

問題75のパディングの処理を活用して、ミニバッチでモデルを学習せよ。また、学習したモデルの開発セットにおける正解率を求めよ。

In [8]:
import torch
from torch.utils.data import DataLoader, Dataset

device = "cpu"

class SSTDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
    
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        return self.dataset[idx]

# モデルの定義を修正してバッチ処理に対応
class LogisticRegression(nn.Module):
    def __init__(self, embedding_matrix, freeze_embedding=True):
        super().__init__()
        self.embedding = nn.Embedding.from_pretrained(
            torch.tensor(embedding_matrix, dtype=torch.float32),
            freeze=freeze_embedding,
        )
        self.linear = nn.Linear(embedding_matrix.shape[1], 1)
    
    def forward(self, input_ids):
        # [batch_size, seq_len] -> [batch_size, seq_len, emb_dim]
        embedded = self.embedding(input_ids)
        # 各シーケンスの平均ベクトルを計算 [batch_size, emb_dim]
        # パディングを無視するためのマスク作成
        mask = (input_ids != 0).float().unsqueeze(-1)  # [batch_size, seq_len, 1]
        sum_embeddings = torch.sum(embedded * mask, dim=1)  # [batch_size, emb_dim]
        sum_mask = torch.sum(mask, dim=1)  # [batch_size, 1]
        mean_embeddings = sum_embeddings / sum_mask  # [batch_size, emb_dim]
        
        return self.linear(mean_embeddings)  # [batch_size, 1]

# データローダーの作成
batch_size = 32
train_dataset_obj = SSTDataset(train_dataset)
dev_dataset_obj = SSTDataset(dev_dataset)

train_loader = DataLoader(
    train_dataset_obj, 
    batch_size=batch_size, 
    shuffle=True, 
    collate_fn=collate
)

dev_loader = DataLoader(
    dev_dataset_obj, 
    batch_size=batch_size, 
    shuffle=False, 
    collate_fn=collate
)

# モデルの初期化
model = LogisticRegression(embedding_matrix, freeze_embedding=True).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.BCEWithLogitsLoss()

# 学習
def train_batch(model, data_loader, optimizer, criterion, device):
    model.train()
    total_loss = 0.0
    
    for batch in data_loader:
        input_ids = batch["input_ids"].to(device)
        labels = batch["label"].to(device)
        
        optimizer.zero_grad()
        outputs = model(input_ids)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item() * input_ids.size(0)
    
    return total_loss / len(data_loader.dataset)

# 評価
def evaluate_batch(model, data_loader, device):
    model.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch["input_ids"].to(device)
            labels = batch["label"].to(device)
            
            outputs = model(input_ids)
            predictions = (torch.sigmoid(outputs) > 0.5).float()
            
            correct += (predictions == labels).sum().item()
            total += labels.size(0)
    
    return correct / total

num_epochs = 5
for epoch in range(num_epochs):
    train_loss = train_batch(model, train_loader, optimizer, criterion, device)
    train_acc = evaluate_batch(model, train_loader, device)
    dev_acc = evaluate_batch(model, dev_loader, device)
    
    print(f"Epoch {epoch + 1}/{num_epochs}")
    print(f"  Training Loss: {train_loss:.4f}")
    print(f"  Training Accuracy: {train_acc:.4f}")
    print(f"  Validation Accuracy: {dev_acc:.4f}")

# 最終的な評価
final_accuracy = evaluate_batch(model, dev_loader, device)
print(f"Final Validation Accuracy: {final_accuracy:.4f}")

torch.save(model.state_dict(), "./model/No_76.pt")

Epoch 1/5
  Training Loss: 0.4942
  Training Accuracy: 0.8247
  Validation Accuracy: 0.7718
Epoch 2/5
  Training Loss: 0.4063
  Training Accuracy: 0.8336
  Validation Accuracy: 0.7798
Epoch 3/5
  Training Loss: 0.3881
  Training Accuracy: 0.8382
  Validation Accuracy: 0.7867
Epoch 4/5
  Training Loss: 0.3805
  Training Accuracy: 0.8399
  Validation Accuracy: 0.7913
Epoch 5/5
  Training Loss: 0.3763
  Training Accuracy: 0.8415
  Validation Accuracy: 0.7993
Final Validation Accuracy: 0.7993


## 77. GPU上での学習

問題76のモデル学習をGPU上で実行せよ。また、学習したモデルの開発セットにおける正解率を求めよ。

In [9]:
import torch
from torch.utils.data import DataLoader, Dataset

# devide = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class SSTDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        return self.dataset[idx]


# モデルの定義を修正してバッチ処理に対応
class LogisticRegression(nn.Module):
    def __init__(self, embedding_matrix, freeze_embedding=True):
        super().__init__()
        self.embedding = nn.Embedding.from_pretrained(
            torch.tensor(embedding_matrix, dtype=torch.float32),
            freeze=freeze_embedding,
        )
        self.linear = nn.Linear(embedding_matrix.shape[1], 1)

    def forward(self, input_ids):
        # [batch_size, seq_len] -> [batch_size, seq_len, emb_dim]
        embedded = self.embedding(input_ids)
        # 各シーケンスの平均ベクトルを計算 [batch_size, emb_dim]
        # パディングを無視するためのマスク作成
        mask = (input_ids != 0).float().unsqueeze(-1)  # [batch_size, seq_len, 1]
        sum_embeddings = torch.sum(embedded * mask, dim=1)  # [batch_size, emb_dim]
        sum_mask = torch.sum(mask, dim=1)  # [batch_size, 1]
        mean_embeddings = sum_embeddings / sum_mask  # [batch_size, emb_dim]

        return self.linear(mean_embeddings)  # [batch_size, 1]


# データローダーの作成
batch_size = 32
train_dataset_obj = SSTDataset(train_dataset)
dev_dataset_obj = SSTDataset(dev_dataset)

train_loader = DataLoader(
    train_dataset_obj, batch_size=batch_size, shuffle=True, collate_fn=collate
)

dev_loader = DataLoader(
    dev_dataset_obj, batch_size=batch_size, shuffle=False, collate_fn=collate
)

# モデルの初期化
model = LogisticRegression(embedding_matrix, freeze_embedding=True).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.BCEWithLogitsLoss()


# 学習
def train_batch(model, data_loader, optimizer, criterion, device):
    model.train()
    total_loss = 0.0

    for batch in data_loader:
        input_ids = batch["input_ids"].to(device)
        labels = batch["label"].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        total_loss += loss.item() * input_ids.size(0)

    return total_loss / len(data_loader.dataset)


# 評価
def evaluate_batch(model, data_loader, device):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch["input_ids"].to(device)
            labels = batch["label"].to(device)

            outputs = model(input_ids)
            predictions = (torch.sigmoid(outputs) > 0.5).float()

            correct += (predictions == labels).sum().item()
            total += labels.size(0)

    return correct / total


num_epochs = 5
for epoch in range(num_epochs):
    train_loss = train_batch(model, train_loader, optimizer, criterion, device)
    train_acc = evaluate_batch(model, train_loader, device)
    dev_acc = evaluate_batch(model, dev_loader, device)

    print(f"Epoch {epoch + 1}/{num_epochs}")
    print(f"  Training Loss: {train_loss:.4f}")
    print(f"  Training Accuracy: {train_acc:.4f}")
    print(f"  Validation Accuracy: {dev_acc:.4f}")

# 最終的な評価
final_accuracy = evaluate_batch(model, dev_loader, device)
print(f"Final Validation Accuracy: {final_accuracy:.4f}")

torch.save(model.state_dict(), "./model/No_77.pt")

Epoch 1/5
  Training Loss: 0.4944
  Training Accuracy: 0.8277
  Validation Accuracy: 0.7810
Epoch 2/5
  Training Loss: 0.4064
  Training Accuracy: 0.8344
  Validation Accuracy: 0.7890
Epoch 3/5
  Training Loss: 0.3882
  Training Accuracy: 0.8374
  Validation Accuracy: 0.7821
Epoch 4/5
  Training Loss: 0.3804
  Training Accuracy: 0.8371
  Validation Accuracy: 0.7821


KeyboardInterrupt: 

## 78. 単語埋め込みのファインチューニング

問題77の学習において、単語埋め込みのパラメータも同時に更新するファインチューニングを導入せよ。また、学習したモデルの開発セットにおける正解率を求めよ。

In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset

# devide = torch.device("cuda" if torch.cuda.is_available() else "cpu")


class SSTDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        return self.dataset[idx]


class LogisticRegression(nn.Module):
    # freeze_embedding を False にすると埋め込み層の重みを学習する
    def __init__(self, embedding_matrix, freeze_embedding=False):
        super().__init__()
        self.embedding = nn.Embedding.from_pretrained(
            torch.tensor(embedding_matrix, dtype=torch.float32),
            freeze=freeze_embedding,
        )
        self.linear = nn.Linear(embedding_matrix.shape[1], 1)

    def forward(self, input_ids):
        # [batch_size, seq_len] -> [batch_size, seq_len, emb_dim]
        embedded = self.embedding(input_ids)
        # 各シーケンスの平均ベクトルを計算 [batch_size, emb_dim]
        # パディングを無視するためのマスク作成
        mask = (input_ids != 0).float().unsqueeze(-1)  # [batch_size, seq_len, 1]
        sum_embeddings = torch.sum(embedded * mask, dim=1)  # [batch_size, emb_dim]
        sum_mask = torch.sum(mask, dim=1)  # [batch_size, 1]
        mean_embeddings = sum_embeddings / sum_mask  # [batch_size, emb_dim]

        return self.linear(mean_embeddings)  # [batch_size, 1]


# データローダーの作成
batch_size = 32
train_dataset_obj = SSTDataset(train_dataset)
dev_dataset_obj = SSTDataset(dev_dataset)

train_loader = DataLoader(
    train_dataset_obj, batch_size=batch_size, shuffle=True, collate_fn=collate
)

dev_loader = DataLoader(
    dev_dataset_obj, batch_size=batch_size, shuffle=False, collate_fn=collate
)

# モデルの初期化
model = LogisticRegression(embedding_matrix, freeze_embedding=True).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.BCEWithLogitsLoss()


# 学習
def train_batch(model, data_loader, optimizer, criterion, device):
    model.train()
    total_loss = 0.0

    for batch in data_loader:
        input_ids = batch["input_ids"].to(device)
        labels = batch["label"].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        total_loss += loss.item() * input_ids.size(0)

    return total_loss / len(data_loader.dataset)


# 評価
def evaluate_batch(model, data_loader, device):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch["input_ids"].to(device)
            labels = batch["label"].to(device)

            outputs = model(input_ids)
            predictions = (torch.sigmoid(outputs) > 0.5).float()

            correct += (predictions == labels).sum().item()
            total += labels.size(0)

    return correct / total


num_epochs = 5
for epoch in range(num_epochs):
    train_loss = train_batch(model, train_loader, optimizer, criterion, device)
    train_acc = evaluate_batch(model, train_loader, device)
    dev_acc = evaluate_batch(model, dev_loader, device)

    print(f"Epoch {epoch + 1}/{num_epochs}")
    print(f"  Training Loss: {train_loss:.4f}")
    print(f"  Training Accuracy: {train_acc:.4f}")
    print(f"  Validation Accuracy: {dev_acc:.4f}")

# 最終的な評価
final_accuracy = evaluate_batch(model, dev_loader, device)
print(f"Final Validation Accuracy: {final_accuracy:.4f}")

torch.save(model.state_dict(), "./model/No_77.pt")

Epoch 1/5
  Training Loss: 0.4953
  Training Accuracy: 0.8270
  Validation Accuracy: 0.7798
Epoch 2/5
  Training Loss: 0.4066
  Training Accuracy: 0.8340
  Validation Accuracy: 0.7913
Epoch 3/5
  Training Loss: 0.3883
  Training Accuracy: 0.8383
  Validation Accuracy: 0.7890
Epoch 4/5
  Training Loss: 0.3806
  Training Accuracy: 0.8404
  Validation Accuracy: 0.7970
Epoch 5/5
  Training Loss: 0.3764
  Training Accuracy: 0.8407
  Validation Accuracy: 0.7936
Final Validation Accuracy: 0.7936


## 79. アーキテクチャの変更

ニューラルネットワークのアーキテクチャを自由に変更し、モデルを学習せよ。また、学習したモデルの開発セットにおける正解率を求めよ。例えば、テキストの特徴ベクトル（単語埋め込みの平均ベクトル）に対して多層のニューラルネットワークを通したり、畳み込みニューラルネットワーク（CNN; Convolutional Neural Network）や再帰型ニューラルネットワーク（RNN; Recurrent Neural Network）などのモデルの学習に挑戦するとよい。

In [11]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


class SSTDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        return self.dataset[idx]


class RNN(nn.Module):
    def __init__(self, embedding_matrix, hidden_dim, freeze_embedding=False):
        super().__init__()
        self.embedding = nn.Embedding.from_pretrained(
            torch.tensor(embedding_matrix, dtype=torch.float32),
            freeze=freeze_embedding,
        )
        emb_dim = embedding_matrix.shape[1]
        self.l1 = nn.RNN(emb_dim, hidden_dim, nonlinearity="tanh", batch_first=True)
        self.l2 = nn.Linear(hidden_dim, 1)
        nn.init.xavier_normal_(self.l1.weight_ih_l0)
        nn.init.orthogonal_(self.l1.weight_hh_l0)

    def forward(self, input_ids):
        # [batch_size, seq_len] -> [batch_size, seq_len, emb_dim]
        embedded = self.embedding(input_ids)
        
        # RNNの処理
        h, _ = self.l1(embedded)
        # 最後のタイムステップの隠れ状態を使用
        y = self.l2(h[:, -1])
        return y


# データローダーの作成
batch_size = 32
train_dataset_obj = SSTDataset(train_dataset)
dev_dataset_obj = SSTDataset(dev_dataset)

train_loader = DataLoader(
    train_dataset_obj, batch_size=batch_size, shuffle=True, collate_fn=collate
)

dev_loader = DataLoader(
    dev_dataset_obj, batch_size=batch_size, shuffle=False, collate_fn=collate
)

# モデルの初期化
model = RNN(embedding_matrix, hidden_dim=128, freeze_embedding=True).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.BCEWithLogitsLoss()


# 学習
def train_batch(model, data_loader, optimizer, criterion, device):
    model.train()
    total_loss = 0.0

    for batch in data_loader:
        input_ids = batch["input_ids"].to(device)
        labels = batch["label"].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        total_loss += loss.item() * input_ids.size(0)

    return total_loss / len(data_loader.dataset)


# 評価
def evaluate_batch(model, data_loader, device):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch["input_ids"].to(device)
            labels = batch["label"].to(device)

            outputs = model(input_ids)
            predictions = (torch.sigmoid(outputs) > 0.5).float()

            correct += (predictions == labels).sum().item()
            total += labels.size(0)

    return correct / total


num_epochs = 5
for epoch in range(num_epochs):
    train_loss = train_batch(model, train_loader, optimizer, criterion, device)
    train_acc = evaluate_batch(model, train_loader, device)
    dev_acc = evaluate_batch(model, dev_loader, device)

    print(f"Epoch {epoch + 1}/{num_epochs}")
    print(f"  Training Loss: {train_loss:.4f}")
    print(f"  Training Accuracy: {train_acc:.4f}")
    print(f"  Validation Accuracy: {dev_acc:.4f}")

# 最終的な評価
final_accuracy = evaluate_batch(model, dev_loader, device)
print(f"Final Validation Accuracy: {final_accuracy:.4f}")

torch.save(model.state_dict(), "./model/No_79.pt")


Epoch 1/5
  Training Loss: 0.6799
  Training Accuracy: 0.5492
  Validation Accuracy: 0.5115
Epoch 2/5
  Training Loss: 0.6874
  Training Accuracy: 0.5569
  Validation Accuracy: 0.5229
Epoch 3/5
  Training Loss: 0.6877
  Training Accuracy: 0.5582
  Validation Accuracy: 0.5092
Epoch 4/5
  Training Loss: 0.6884
  Training Accuracy: 0.5582
  Validation Accuracy: 0.5092
Epoch 5/5
  Training Loss: 0.6891
  Training Accuracy: 0.5582
  Validation Accuracy: 0.5092
Final Validation Accuracy: 0.5092
